The purpose of this notebook is to experiment with various CPU players of the game of canary. 

In canary_notebook.ipynb, the players make random choices. For example, the act of choosing 3 face up cards is done randomly. Another example is every time a player chooses what card to play, they choose randomly from the legal plays in their hand. 

The first strategy I will implement is the 'play low' strategy. In this strategy, the player will always attempt to play their lowest valued playable card. For example, suppose the top card is a 4 and the current player has a 3, 4, and 5. Using the 'play low' strategy the CPU will choose to play the 4 since it is the lowest valued playable card. Special cards will be treated as the most valuable cards, with the 2 being the highest value card, since having the ability to play again allows for certain face up strategies. The 10 is the next most valuable with its ability to clear potentially large pick ups that would destroy all hopes of winning. The Joker is strong in end game situations where coordinated play between players can result in a leading player not being able to play their last card(s). 

These assumptions about card values are purely based on my experience playing many games with my friends.

In [4]:
import itertools
import random
import json
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from itertools import groupby
import numpy as np
from collections import defaultdict
import time
from sklearn.ensemble import RandomForestClassifier
import pickle
import os

C:\Users\ellio\AppData\Local\Temp\ipykernel_21812\3413549242.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        # Determine if the card is special
        self.is_special = value in [2, 10, 'Joker']
        self.is_seven = value in [7]

    def __repr__(self):
        return f"{self.value} {self.suit}" if self.suit else f"{self.value}"

In [6]:
class CardGame:
    def __init__(self, num_players, game_number):
        self.num_players = num_players
        self.deck = create_deck(self)
        self.players_faceup_strategy = [[] for _ in range(num_players)]
        self.regular_play_strategies = [[] for _ in range(num_players)]
        self.players_hands = [[] for _ in range(num_players)]
        self.face_down_cards = [[] for _ in range(num_players)]
        self.face_up_cards = [[] for _ in range(num_players)]
        self.draw_pile = []
        self.play_pile = []
        self.game_over = False
        self.play_direction = 1 # 1 for clockwise, -1 for counterclockwise
        self.turn = 0
        self.known_opponent_cards = [[] for _ in range(num_players)]
        self.game_number = game_number
        self.winner = None
        setup_game(self)
        self.current_player_index = find_starting_player(self)
        self.playable_cards = [self.players_hands[self.current_player_index]]
        start_game(self)
    

In [7]:
def create_deck(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        values = [2,3,4,5,6,7,8,9,10,11,12,13,14]
        deck = [Card(value, suit) for suit in suits for value in values]

        # Include Jokers without a suit
        deck.extend([Card('Joker', None) for _ in range(2)])  # Adding two Jokers

        # Use 2 decks for 5-8 players
        if self.num_players >= 5:
            deck = deck * 2

        #print(deck)

        return deck

In [8]:
def find_starting_player(self):
        # Define the order for determining the starting player
        card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 'Joker', 10, 2]

        # Initialize counts for each card value for each player
        player_card_counts = [{value: 0 for value in card_value_order} for _ in range(self.num_players)]

        # Count the number of each card value in each player's hand
        for player_index, hand in enumerate(self.players_hands):
            for card in hand:
                if card.value in card_value_order:
                    player_card_counts[player_index][card.value] += 1

        # Determine who starts based on the cards
        for value in card_value_order:
            # Find players with the current card value
            players_with_card = [(index, count[value]) for index, count in enumerate(player_card_counts) if count[value] > 0]

            if not players_with_card:  # If no player has the card, continue to the next value
                continue

            # If only one player has the card, they start
            if len(players_with_card) == 1:
                return players_with_card[0][0]

            # If multiple players have the same card, compare counts
            max_count = max(players_with_card, key=lambda x: x[1])[1]
            players_with_max = [player for player, count in players_with_card if count == max_count]

            if len(players_with_max) == 1:
                return players_with_max[0]
            # If still tied, continue to the next card value

        # If somehow no one can start, return a random player as fallback
        return random.randint(0, self.num_players - 1)

In [9]:
def setup_game(self):
        # Shuffle the deck
        random.shuffle(self.deck) 

        # Deal 3 face-down and 6 to choose face-up cards to each player
        for i in range(self.num_players):

            Faceup_strategies = ['Play High']
            self.players_faceup_strategy[i] = random.choice(Faceup_strategies)

            Regular_play_strategies = ['Play Low','Sabotage']
            self.regular_play_strategies[i] = random.choice(Regular_play_strategies)
            #print(self.players_faceup_strategy[i])

            # Deal 3 face-down cards
            self.face_down_cards[i] = [self.deck.pop() for _ in range(3)]

            # Deal 6 cards for choosing 3 to be face-up. These will be passed to the AI
            face_up_candidates = [self.deck.pop() for _ in range(6)]


            # Players choose 3 of these to be face-up, for simplicity, randomly select here
            self.face_up_cards[i] = CPU_face_up(face_up_candidates, type=self.players_faceup_strategy[i])
            
            # The remaining 3 cards from the 6 initially dealt for choosing go into the player's hand
            remaining_cards_counter = Counter(face_up_candidates) - Counter(self.face_up_cards[i])
            self.players_hands[i] = list(remaining_cards_counter.elements())

            #Error testing
            #print("Candidates: ", face_up_candidates)
            #print("Face Up: ", self.face_up_cards[i])
            #print("Hand : ", self.players_hands[i])

        # The rest of the cards form the draw pile
        self.draw_pile = self.deck

In [10]:
def CPU_face_up(cards, type = 'Random'):
    if type == 'Random':
        return  random.sample(cards,3)
    if type == "Double Up":
        return double_up(cards)
    if type == "Play High":
        return play_high(cards)

In [11]:
def double_up(cards):
    # Sort the cards based on the defined order
    card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 'Joker', 10, 2]
    cards.sort(key=lambda card: card_value_order.index(card.value), reverse=True)

    # Find the number of 2's and identify pairs
    twos = [card for card in cards if card.value == 2]
    pairs = {}
    for card in cards:
        if card.value != 2:
            if card.value in pairs:
                pairs[card.value].append(card)
            else:
                pairs[card.value] = [card]

    # Return cards based on the conditions
    if len(twos) == 1 and any(len(pair) == 2 for pair in pairs.values()):
        # Find the highest value pair
        highest_pair = max([pair for pair in pairs.values() if len(pair) == 2], key=lambda pair: card_value_order.index(pair[0].value))
        return twos + highest_pair
    else:
        return twos[:3] if len(twos) >= 3 else twos + cards[len(twos):3]

In [12]:
def play_high(cards):
    card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 'Joker', 10, 2]
    sorted_cards = sorted(cards, key=lambda card: card_value_order.index(card.value))
    return sorted_cards[3:]

In [13]:
def start_game(self):
    while not self.game_over:
        # Player takes their turn.
        game_data = collect_game_data(self)
        
        train_Q_model()
        #print(game_data)

        self.game_over = check_game_over(self)
        
        play_turn(self)

        


    collect_game_data(self)

In [14]:
import json

def calculate_reward(old_state, new_state):
    # Check if the states are non-empty strings and convert them to dictionaries if necessary
    if isinstance(old_state, str) and old_state.strip():
        old_state = json.loads(old_state)
    elif old_state is None or old_state == "":
        return 0  # Return a default reward if the state is None or empty

    if isinstance(new_state, str) and new_state.strip():
        new_state = json.loads(new_state)
    elif new_state is None or new_state == "":
        return 0  # Return a default reward if the state is None or empty

    # Get the index of the player who just played
    player_index = old_state['current_player'] - 1

    # Calculate the change in the number of face-up cards and cards in hand for the player
    change_in_face_up_cards = old_state['num_face_up_cards'][player_index] - new_state['num_face_up_cards'][player_index]
    change_in_cards_in_hand = old_state['num_cards_in_hands'][player_index] - new_state['num_cards_in_hands'][player_index]

    # Define the reward based on the changes
    reward = change_in_face_up_cards + change_in_cards_in_hand

    return reward


In [15]:
def train_Q_model():
    # Load the Q-table and other data from the pickle file
    data_file = 'q_learning_data.pkl'
    if os.path.exists(data_file):
        with open(data_file, 'rb') as file:
            data = pickle.load(file)
        q_table = data['q_table']
        state_index_history = data['state_index_history']
        action_index_history = data['action_index_history']
    else:
        # Initialize new data if the file does not exist
        q_table = {}
        state_index_history = []
        action_index_history = []

    # Define the learning parameters
    alpha = 0.1  # Learning rate
    gamma = 0.99  # Discount factor

    # Example usage of the loaded data
    file_path = 'game_data.txt'
    game_data = read_game_data(file_path)
    extracted_data = extract_features(game_data)

    # Check if there are at least 2 lines of game data
    if len(extracted_data) >= 2:
        # Check if the turn number of the new state is 0
        if extracted_data[-1]['turn_number'] == 0:
            return  # End the function if the turn number is 0

        old_state = extracted_data[-2]  # Get the second to last line of game data
        new_state = extracted_data[-1]  # Get the last line of game data
        reward = calculate_reward(old_state, new_state)
        new_state_index = convert_data_to_index(new_state)
        new_num_actions = len(new_state['playable_cards'])

        # Initialize the Q-values for the new state if it's not in the Q-table
        if new_state_index not in q_table:
            q_table[new_state_index] = np.zeros(new_num_actions)
        else:
            # Resize the Q-values array if necessary
            if len(q_table[new_state_index]) < new_num_actions:
                q_table[new_state_index] = np.resize(q_table[new_state_index], new_num_actions)

        # Proceed only if action_index_history and state_index_history are not empty
        if action_index_history and state_index_history:
            # Get the last action index and corresponding state index
            action_index = action_index_history[-1]
            state_index = state_index_history[-1]

            # Check if action_index is within the valid range for the Q-values array
            if action_index < len(q_table[state_index]):
                # Update Q-value
                q_table[state_index][action_index] = (
                    q_table[state_index][action_index] +
                    alpha * (reward + gamma * np.max(q_table[new_state_index]) - q_table[state_index][action_index])
                )

                # Save the updated data
                data = {
                    'q_table': q_table,
                    'state_index_history': state_index_history,
                    'action_index_history': action_index_history
                }
                with open(data_file, 'wb') as file:
                    pickle.dump(data, file)

In [16]:
def collect_game_data(self):
        game_data = {
            'game_number': self.game_number,
            'turn_number': self.turn,
            'current_player': self.current_player_index + 1,
            'num_players': self.num_players,
            'cards_in_hands': [[str(card) for card in hand] if i == self.current_player_index else ['Unknown'] * len(hand) for i, hand in enumerate(self.players_hands)],
            'playable_cards' : str(self.playable_cards),
            'known_opponent_cards': [[str(card) for card in known_cards] for known_cards in self.known_opponent_cards], 
            'num_cards_in_hands': [len(hand) for hand in self.players_hands],
            'top_card': str(self.play_pile[-1]) if self.play_pile else None,
            'cards_in_pile': [str(card) for card in self.play_pile],
            'face_up_cards': [[str(card) for card in face_up] for face_up in self.face_up_cards],
            'num_face_up_cards': [len(face_up) for face_up in self.face_up_cards],
            'num_face_down_cards': [len(face_down) for face_down in self.face_down_cards],
            'play_direction': self.play_direction,
            'num_cards_in_draw_pile': len(self.draw_pile),
            'game_over': self.game_over,
            'players_faceup_strategy' : self.players_faceup_strategy,
            'regular_play_strategies' : self.regular_play_strategies,
            'winner' : self.winner
        }

        with open('game_data.txt', 'a') as file:
            json.dump(game_data, file)
            file.write('\n\n')

        return json.dumps(game_data)

In [17]:
def check_game_over(self):
        # Implement logic to determine if the game has ended.
        # This could involve checking if any player has successfully played all their cards (hand, face-up, and face-down).
        for player_index in range(self.num_players):
            if (not self.players_hands[player_index] and 
                not self.face_up_cards[player_index] and 
                not self.face_down_cards[player_index]):
                #print(f"Player {player_index + 1} wins and is declared the Canary!")
                self.winner = player_index+1
                return True  # A winner is found
        return False  # The game continues

In [18]:
def play_turn(self):
    self.turn += 1
    #print(f"Player {self.current_player_index + 1}'s turn:")
        

    # Try playing from hand, face-up, then face-down in order.
    if not attempt_play_from_hand(self):
        if not attempt_play_from_face_up(self):
            if not attempt_play_from_face_down(self):
                return 

In [19]:
def attempt_play_from_hand(self):
        #print("Player Hand: ", self.players_hands[self.current_player_index])
        return attempt_play(self, card_source = self.players_hands[self.current_player_index])

In [20]:
def attempt_play_from_face_up(self):
        #print("Players Face-Up Hand: ", self.face_up_cards[self.current_player_index])
        return attempt_play(self, card_source = self.face_up_cards[self.current_player_index], is_face_up = True)

In [21]:
def attempt_play_from_face_down(self, is_face_down = True):
        # Face-down play is a bit different as it involves random choice and immediate play without checking
        if self.face_down_cards[self.current_player_index]:
            chosen_card = random.choice(self.face_down_cards[self.current_player_index])
            ##print("Players Face-Down Hand:: ", chosen_card)
            return attempt_play(self, card_source = chosen_card, is_face_down = True)
        else:
            return False

In [22]:
def attempt_play(self, card_source, is_face_up = False, is_face_down = False):

        if is_face_up:
            if not card_source:
                #Face up cards is empty, move to facedown
                return False
            
            playable_cards = [card for card in card_source if can_play_card(self, card)]
            #print("Playable Cards: ", playable_cards)
            if not playable_cards:
                    #print("No playable cards in face up pile")
                    pick_up_pile(self)
                    return True
            chosen_card = CPU(self, playable_cards, type = self.regular_play_strategies[self.current_player_index])
            play_card(self, chosen_card, is_face_up = True)
            return True
        
        if is_face_down:
            if not card_source:
                return False
            
            #print("Card Selected: ", card_source)
            
            if can_play_card(self, card_source):
                play_card(self, card_source, is_face_down=True)
                return True
            else:
                self.play_pile.append(card_source)
                self.face_down_cards[self.current_player_index].remove(card_source)
                pick_up_pile(self)
                return True

        
        if not card_source:
            #print("No Cards in Hand!")
            return False
        
        playable_cards = [card for card in card_source if can_play_card(self,card)]
        #print("Playable Cards: ", playable_cards)
        if not playable_cards:
            pick_up_pile(self)   
            return True

        chosen_card = CPU(self, playable_cards, type = self.regular_play_strategies[self.current_player_index])
        play_card(self, chosen_card)
        return True

In [23]:
def can_play_card(self, card):
        """Check if the card can be played on top of the play pile."""
        if not self.play_pile:
            return True  # Any card can be played if the play pile is empty
        
        top_card = self.play_pile[-1]
        
        # Allow any card to be played if the top card is a special card or the played card is special
        if top_card.is_special:
            return True
        
        if card.is_special:
            return True
        
        if top_card.is_seven:
            #print(card.value, " <= ", top_card.value,"?", card.value <= top_card.value)
            return card.value <= 7
        
        #print(card.value, " >= ", top_card.value,"?", card.value >= top_card.value)
        return card.value >= top_card.value

In [24]:
def pick_up_pile(self):
        """Player picks up the play pile."""
        self.players_hands[self.current_player_index].extend(self.play_pile)
        self.known_opponent_cards[self.current_player_index].extend(self.play_pile)
        self.play_pile.clear()
        post_play_card_actions(self)
        #print("Pile picked up!")

In [25]:
def post_play_card_actions(self):
        # Ensures the player has at least 3 cards in their hand if the draw pile isn't empty
        while len(self.players_hands[self.current_player_index]) < 3 and self.draw_pile:
            draw_card(self)
            #print(f"Player {self.current_player_index + 1} draws a card. Hand now: {self.players_hands[self.current_player_index]}")
        
        if len(self.play_pile) >= 4:
                #print("thats a big pile")
                for i in range(len(self.play_pile) -3):
                    if self.play_pile[i] == self.play_pile[i+1] == self.play_pile[i+2] == self.play_pile[i+3]:
                        #print("4 in a row!")    
                        self.play_pile.clear()
                
            
        # Move to the next player
        self.current_player_index = (self.current_player_index + self.play_direction) % self.num_players   

In [26]:
def draw_card(self):
        """Player draws a card from the draw pile."""
        if self.draw_pile:
            card_drawn = self.draw_pile.pop()
            self.players_hands[self.current_player_index].append(card_drawn)
        else:
            print("The draw pile is empty. No card drawn.")

In [27]:
def CPU(self, cards, type = 'Random'):
    if type =='Random':
        return get_random_card(cards)
    if type == 'Play Low':
        return play_low_decision_strategy(self, cards)
    if type == 'Sabotage':
        return sabotage(self, cards)
    if type == 'Play High':
        return play_high_decision_strategy(self,cards)

In [28]:
def get_random_card(cards):
    return random.choice(cards)

In [29]:
def play_low_decision_strategy(self, cards):
    card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 'Joker', 10, 2]
    
    # Define a custom sorting key based on the card value order
    def sort_key(card):
        return card_value_order.index(card.value)

    # Sort the cards based on their value using the custom key
    sorted_cards = sorted(cards, key=sort_key)
    
    return sorted_cards[0]


In [30]:
def play_high_decision_strategy(self, cards):
    card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 'Joker', 10, 2]
    
    # Define a custom sorting key based on the card value order
    def sort_key(card):
        return card_value_order.index(card.value)

    # Sort the cards based on their value using the custom key
    sorted_cards = sorted(cards, key=sort_key)
    
    return sorted_cards[-1]

In [31]:
def average_expected_draw_value(self, face_up_cards, hand, known_cards):
    card_lists = [face_up_cards, hand, known_cards]
    print(card_lists)
    card_value_order = [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 'Joker', 10, 2]
    initial_count = {value: 4 for value in card_value_order[:-1]}
    initial_count['Joker'] = 2
    initial_count[2] = 4

    # Flatten the list of lists and count the values
    all_cards = [card.value for sublist in card_lists for card in sublist]
    card_counts = Counter(all_cards)

    # Subtract the counts from the initial count
    for value, count in card_counts.items():
        initial_count[value] -= count

    # Sort the count dictionary according to the card value order
    sorted_count = {value: initial_count[value] for value in card_value_order}

    # Multiply counts by win percentages
    card_win_percentage = [0.259, 0.33, 0.4, 0.48, 0.55, 0.55, 0.62, 0.7, 0.77, 0.85, 0.92, 1, 1, 1]
    for i, value in enumerate(card_value_order):
        sorted_count[value] *= card_win_percentage[i]

    # Calculate the total count of cards
    total_count = sum(initial_count.values())

    # Calculate the average value of sorted_count
    average_value = sum(sorted_count.values()) / total_count

    # Example usage
    

    return average_value

In [32]:
def sabotage(self, cards):
    action = Q_learning(cards)
    return action
    

In [33]:
def Q_learning(cards):
    # Initialize the Q-table and other data as a dictionary
    data_file = 'q_learning_data.pkl'
    if os.path.exists(data_file):
        # Load the Q-table and other data from the file
        with open(data_file, 'rb') as file:
            data = pickle.load(file)
        q_table = data['q_table']
        state_index_history = data['state_index_history']
        action_index_history = data['action_index_history']
    else:
        # Initialize a new Q-table as a dictionary
        q_table = {}
        state_index_history = []
        action_index_history = []

    # Define the learning parameters
    epsilon = 0.1  # Exploration rate

    file_path = 'game_data.txt'


    # Q-learning algorithm
    game_data = read_game_data(file_path)
    extracted_data = extract_features(game_data)
    line_number = len(extracted_data) - 1  # Get the last line of extracted data
    state = get_line_from_extracted_data(extracted_data, line_number)
    state_index = convert_data_to_index(state)
    num_actions = len(cards)  # Number of actions depends on the 'cards' argument

    # Initialize the Q-values for the new state if it's not in the Q-table
    if state_index not in q_table:
        q_table[state_index] = np.zeros(num_actions)

    if np.random.uniform(0, 1) < epsilon:
        action_index = np.random.choice(range(num_actions))  # Explore action space
    else:
        action_index = np.argmax(q_table[state_index])  # Exploit learned values

    # Save state_index and action_index
    state_index_history.append(state_index)
    action_index_history.append(action_index)

    # Save the updated Q-table and other data
    data = {
        'q_table': q_table,
        'state_index_history': state_index_history,
        'action_index_history': action_index_history
    }
    with open(data_file, 'wb') as file:
        pickle.dump(data, file)

    return cards[action_index]  # Return the selected action as a card object


In [34]:
def read_game_data(file_path):
    game_data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                game_state = json.loads(line.strip())
                game_data.append(game_state)
            except json.JSONDecodeError:
                continue  # Skip lines that can't be decoded as JSON
    return game_data

In [35]:
def extract_features(game_data):
    extracted_data = []
    for state in game_data:
        new_state = {
            "turn_number": state["turn_number"],
            "current_player": state["current_player"],
            "num_players": state["num_players"],
            "cards_in_hands": state["cards_in_hands"],
            "playable_cards": state["playable_cards"],
            "known_opponent_cards": state["known_opponent_cards"],
            "num_cards_in_hands": state["num_cards_in_hands"],
            "top_card": state["top_card"],
            "cards_in_pile": state["cards_in_pile"],
            "face_up_cards": state["face_up_cards"],
            "num_face_up_cards": state["num_face_up_cards"],
            "num_face_down_cards": state["num_face_down_cards"],
            "play_direction": state["play_direction"],
            "num_cards_in_draw_pile": state["num_cards_in_draw_pile"]
        }
        extracted_data.append(new_state)
    return extracted_data

In [36]:
def get_line_from_extracted_data(extracted_data, line_number):
    if line_number < 0 or line_number >= len(extracted_data):
        return "Line number out of range"
    return extracted_data[line_number]

In [37]:
def parse_playable_cards(card_string):
    # Remove the brackets and split the string by commas
    card_list = card_string.strip("[]").split(", ")
    # Strip extra whitespace and add quotes around each element
    return [card.strip().replace(" ", "_") for card in card_list]

In [38]:
def convert_data_to_index(data):
    # Create a unique string representation of the data
    data_string = (
        str(data["turn_number"]) +
        str(data["current_player"]) +
        str(data["num_players"]) +
        "".join(["".join(hand) for hand in data["cards_in_hands"]]) +
        "".join(data["playable_cards"]) +  # Convert the list to a string
        "".join(["".join(map(str, cards)) for cards in data["known_opponent_cards"]]) +
        "".join(map(str, data["num_cards_in_hands"])) +
        (data["top_card"] if data["top_card"] else "None") +
        "".join(data["cards_in_pile"]) +
        "".join(["".join(cards) for cards in data["face_up_cards"]]) +
        "".join(map(str, data["num_face_up_cards"])) +
        "".join(map(str, data["num_face_down_cards"])) +
        str(data["play_direction"]) +
        str(data["num_cards_in_draw_pile"])
    )

    # Hash the string to get a unique index
    data_index = hash(data_string)
    return data_index

In [39]:
def play_card(self, chosen_card, is_face_up=False, is_face_down=False):
        # Remove the card from known_opponent_cards if it was known
        if chosen_card in self.known_opponent_cards[self.current_player_index]:
            self.known_opponent_cards[self.current_player_index].remove(chosen_card)

        # Check for special cards (Joker, 2, 10, and handling of 7 if needed)
        if chosen_card.is_special:
            if chosen_card.value == 'Joker':
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                self.play_direction *= -1
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                else:
                    self.face_down_cards[self.current_player_index].remove(chosen_card)        
                post_play_card_actions(self)
                return
            elif chosen_card.value == 2:
                # Logic for 2 allowing the player to play again
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                else:
                    self.face_down_cards[self.current_player_index].remove(chosen_card) 
                allow_play_again(self)
                post_play_card_actions(self)
                return
            elif chosen_card.value == 10:
                # Maybe a special logic for 10, like clearing the play pile
                self.play_pile.append(chosen_card)
                #print(f"Played {chosen_card}.")
                clear_play_pile(self)
                # Remove the card from its current location unless it's face-down
                if not is_face_down:
                    if is_face_up:
                        self.face_up_cards[self.current_player_index].remove(chosen_card)
                    else:
                        self.players_hands[self.current_player_index].remove(chosen_card)
                else:
                    self.face_down_cards[self.current_player_index].remove(chosen_card) 
                        
                post_play_card_actions(self)
                return
            # Additional logic for any special actions based on card
        elif chosen_card.is_seven:
            # Handle seven's special rule if applicable
            self.play_pile.append(chosen_card)
            #print(f"Played {chosen_card}.")
            # Remove the card from its current location unless it's face-down
            if not is_face_down:
                if is_face_up:
                    self.face_up_cards[self.current_player_index].remove(chosen_card)
                else:
                    self.players_hands[self.current_player_index].remove(chosen_card)
            else:
                self.face_down_cards[self.current_player_index].remove(chosen_card) 
            post_play_card_actions(self)
            return

        if not is_face_down:
            if is_face_up:
                self.face_up_cards[self.current_player_index].remove(chosen_card)
            else:
                self.players_hands[self.current_player_index].remove(chosen_card)
        else:
            self.face_down_cards[self.current_player_index].remove(chosen_card) 

        self.play_pile.append(chosen_card)
        #print(f"Played {chosen_card}.")
        post_play_card_actions(self)

In [40]:
def allow_play_again(self):
        #print("Play again:")
        self.current_player_index = (self.current_player_index - self.play_direction) % self.num_players
        #print(self.current_player_index)
        # Implement logic to allow the current player to play again. This might mean not advancing
        # the current_player_index or setting a flag that allows another turn for the current player.

In [41]:
def clear_play_pile(self):
        #print("Clearing play pile")
        self.play_pile.clear()
        # Implement logic to clear the play pile if a 10 is played, for example.

In [42]:
game_number = 1
while game_number <= 1000:    
    print("Game: ", game_number)
    game = CardGame(4, game_number)
    game_number += 1
    

Game:  1


Game:  2
Game:  3
Game:  4
Game:  5
Game:  6
Game:  7
Game:  8
Game:  9
Game:  10
Game:  11
Game:  12
Game:  13
Game:  14
Game:  15
Game:  16
Game:  17
Game:  18
Game:  19
Game:  20
Game:  21
Game:  22
Game:  23
Game:  24
Game:  25
Game:  26
Game:  27
Game:  28
Game:  29
Game:  30
Game:  31
Game:  32
Game:  33
Game:  34
Game:  35
Game:  36
Game:  37
Game:  38
Game:  39
Game:  40
Game:  41
Game:  42
Game:  43
Game:  44
Game:  45
Game:  46
Game:  47
Game:  48
Game:  49
Game:  50
Game:  51
Game:  52
Game:  53
Game:  54
Game:  55
Game:  56
Game:  57
Game:  58
Game:  59
Game:  60
Game:  61
Game:  62
Game:  63


KeyboardInterrupt: 